In [4]:
import xarray
import os
import pandas as pd
import Preprocessing_test

extractor = Preprocessing_test.FileExtractor()

df_dwd_hornsea = extractor.combine_files("data", "dwd_icon_eu_hornsea")
df_dwd_pes = extractor.combine_files("data", "dwd_icon_eu_pes10")
df_dwd_demand = extractor.combine_files("data", "dwd_icon_eu_demand")

ncep_gfs_hornsea = extractor.combine_files("data", "ncep_gfs_hornsea")
ncep_gfs_pes = extractor.combine_files("data", "ncep_gfs_pes10")
ncep_gfs_demand = extractor.combine_files("data", "ncep_gfs_demand")

import numpy as np
import dask.dataframe as dd
import math

preprocesser = Preprocessing_test.Preprocessing()

df_dwd_hornsea = preprocesser.preprocess_geo_data(df_dwd_hornsea)
ncep_gfs_hornsea = preprocesser.preprocess_geo_data(ncep_gfs_hornsea)
df_dwd_pes = preprocesser.preprocess_geo_data(df_dwd_pes)
ncep_gfs_pes = preprocesser.preprocess_geo_data(ncep_gfs_pes)
df_dwd_demand = preprocesser.preprocess_geo_data(df_dwd_demand)
ncep_gfs_demand = preprocesser.preprocess_geo_data(ncep_gfs_demand)

hornsea = preprocesser.merge_weather_stations_data(df_dwd_hornsea, ncep_gfs_hornsea)
demand = preprocesser.merge_weather_stations_data(df_dwd_demand, ncep_gfs_demand)
pes = preprocesser.merge_weather_stations_data(df_dwd_pes, ncep_gfs_pes)

df_energy = extractor.combine_files("data", "Energy_data", ".csv")
df_energy = preprocesser.preprocess_energy_data(df_energy)

merged_hornsea = preprocesser.merge_geo_energy_outage_data(hornsea, df_energy)
merged_pes = preprocesser.merge_geo_energy_outage_data(pes, df_energy)
merged_demand = preprocesser.merge_geo_energy_outage_data(demand, df_energy)

merged_hornsea = preprocesser.add_difference_features(merged_hornsea)
merged_pes = preprocesser.add_difference_features(merged_pes)
merged_demand = preprocesser.add_difference_features(merged_demand)

In [5]:
import importlib
import Preprocessing_test
importlib.reload(Preprocessing_test)


feature_engineerer_wind = Preprocessing_test.FeatureEngineerer(merged_hornsea, label = 'Wind_MWh_credit')
feature_engineerer_solar = Preprocessing_test.FeatureEngineerer(merged_pes, label = 'Solar_MWh_credit')

In [35]:
from sklearn.utils.fixes import parse_version, sp_version

# This is line is to avoid incompatibility if older SciPy version.
# You should use `solver="highs"` with recent version of SciPy.
solver = "highs" if sp_version >= parse_version("1.6.0") else "interior-point"

from sklearn.linear_model import QuantileRegressor

quantiles = [x for x in np.arange(0.1, 1.0, 0.4)]
q = {}
q["true"] = feature_engineerer_wind.y_test.values
#out_bounds_predictions = np.zeros_like(y_true_mean, dtype=np.bool_)
for quantile in quantiles:
    qr = QuantileRegressor(quantile=quantile, alpha=0, solver=solver)
    y_pred = qr.fit(feature_engineerer_wind.X_train, feature_engineerer_wind.y_train)
    q[str(quantile)] = qr.predict(feature_engineerer_wind.X_test)
    #predictions[quantile] = y_pred


In [58]:
y_pred = pd.DataFrame()
y_pred["true"] = feature_engineerer_wind.y_test

In [71]:
q

{'true': array([373.688, 360.038, 361.818, ...,  85.653,  81.582,  89.046]),
 '0.05': array([156.15321675, 151.57032446, 148.19237557, ..., 151.8650951 ,
        148.02486512, 177.05073289]),
 '0.5': array([236.0797815 , 232.89523571, 234.99558743, ..., 186.10758139,
        180.89298282, 192.81268814]),
 '0.95': array([399.0153393 , 397.44056674, 402.77643325, ..., 381.4866221 ,
        375.02930648, 367.02407186])}

In [6]:
def pinball(y, q, alpha):
    return (y - q) * alpha * (y >= q) + (q - y) * (1 - alpha) * (y < q)

def pinball_score(df, quantiles):
    score = list()
    for qu in quantiles:
        # Berechne den Pinball Loss für jedes Quantil
        score.append(pinball(y=df["true"],
                             q=df[f"{qu}"],
                             alpha=qu/100).mean())
    return sum(score)/len(score)  # Durchschnittlicher Pinball Score

In [72]:
q_df = pd.DataFrame(q)

In [80]:
q_df

,true,0.05,0.5,0.95
0,373.688,156.153217,236.079781,399.015339
1,360.038,151.570324,232.895236,397.440567
2,361.818,148.192376,234.995587,402.776433
3,350.358,151.519207,231.803544,396.017240
4,342.218,147.404771,228.277675,395.516948
...,...,...,...,...
6418,113.759,152.266421,192.167889,389.206420
6419,98.545,151.446493,187.281273,383.433174
6420,85.653,151.865095,186.107581,381.486622
6421,81.582,148.024865,180.892983,375.029306


In [83]:
pinball_score(q_df)

140.61483473774035

In [100]:
from sklearn.utils.fixes import parse_version, sp_version

# This is line is to avoid incompatibility if older SciPy version.
# You should use `solver="highs"` with recent version of SciPy.
solver = "highs" if sp_version >= parse_version("1.6.0") else "interior-point"

from sklearn.linear_model import QuantileRegressor

quantiles = [x for x in np.arange(0.1, 1.0, 0.4)]
q_solar = {}
q_solar["true"] = feature_engineerer_solar.y_test.values
#out_bounds_predictions = np.zeros_like(y_true_mean, dtype=np.bool_)
for quantile in quantiles:
    qr_solar = QuantileRegressor(quantile=quantile, alpha=0, solver=solver)
    qr_solar.fit(feature_engineerer_solar.X_train, feature_engineerer_solar.y_train)
    q_solar[str(quantile)] = qr_solar.predict(feature_engineerer_solar.X_test)
    

In [13]:
qsolar_df = pd.DataFrame(q_solar)

pinball_score(qsolar_df, quantiles)

NameError: name 'q_solar' is not defined

In [9]:
merged_pes.columns

Index(['cloud_cover', 'solar_down_rad', 'temp', 'forecast_horizon',
       'temp_mean', 'temp_std', 'temp_min', 'temp_max', 'solar_down_rad_mean',
       'solar_down_rad_std', 'solar_down_rad_min', 'solar_down_rad_max',
       'cloud_cover_mean', 'cloud_cover_std', 'cloud_cover_min',
       'cloud_cover_max', 'temp_range', 'sin_month', 'cos_month', 'sin_day',
       'cos_day', 'sin_dayofweek', 'cos_dayofweek', 'sin_hour', 'cos_hour',
       'Wind_MWh_credit', 'Solar_MWh_credit', 'temp_diff', 'cloud_cover_diff'],
      dtype='object')

In [7]:
merged_pes_simple = merged_pes[['solar_down_rad', 'Solar_MWh_credit', 'Wind_MWh_credit']]

feature_engineerer_solar_simple = Preprocessing_test.FeatureEngineerer(merged_pes_simple, label = 'Solar_MWh_credit')

from sklearn.utils.fixes import parse_version, sp_version

# This is line is to avoid incompatibility if older SciPy version.
# You should use `solver="highs"` with recent version of SciPy.
solver = "highs" if sp_version >= parse_version("1.6.0") else "interior-point"

from sklearn.linear_model import QuantileRegressor

quantiles = [x for x in np.arange(0.1, 1.0, 0.4)]
q_solar_simple = {}
q_solar_simple["true"] = feature_engineerer_solar_simple.y_test.values
#out_bounds_predictions = np.zeros_like(y_true_mean, dtype=np.bool_)
for quantile in quantiles:
    qr_solar_simple = QuantileRegressor(quantile=quantile, alpha=0, solver=solver)
    qr_solar_simple.fit(feature_engineerer_solar_simple.X_train, feature_engineerer_solar_simple.y_train)
    q_solar_simple[str(quantile)] = qr_solar_simple.predict(feature_engineerer_solar_simple.X_test)
    #predictions[quantile] = y_pred

qsolar_simple_df = pd.DataFrame(q_solar_simple)

In [122]:
qsolar_simple_df

,true,0.1,0.5,0.9
0,0.0,-4.104712,-0.005558,0.001543
1,0.0,-4.104409,-0.005024,0.002288
2,0.0,-4.104902,-0.005893,0.001076
3,0.0,-4.105395,-0.006763,-0.000137
4,0.0,-4.105008,-0.006080,0.000815
...,...,...,...,...
6418,0.0,-4.095159,0.011291,0.025034
6419,0.0,-4.094412,0.012609,0.026871
6420,0.0,-4.095823,0.010121,0.023402
6421,0.0,-4.097234,0.007632,0.019933


In [8]:
pinball_score(qsolar_simple_df, quantiles)

10.431647477457508

### __XGBoost implementation__

In [20]:
import xgboost as xgb

feature_engineerer_solar = Preprocessing_test.FeatureEngineerer(merged_pes, label = 'Solar_MWh_credit')

In [67]:
import xgboost as xgb

feature_engineerer_solar = Preprocessing_test.FeatureEngineerer(merged_pes, label = 'Solar_MWh_credit')

def train_xgboost_model(x_train, y_train, x_test, y_test, alpha):
  evals_result = {}

  Xy_train = xgb.QuantileDMatrix(x_train, y_train)
  Xy_test = xgb.QuantileDMatrix(x_test, y_test, ref=Xy_train)

  booster = xgb.train(
        {
            # Use the quantile objective function.
            "objective": "reg:quantileerror",
            "tree_method": "hist",
            "quantile_alpha": alpha,
            # Let's try not to overfit.
            "learning_rate": 0.04,
            "max_depth": 8,
        },
        Xy_train,
        num_boost_round=250,
        early_stopping_rounds=10,
        evals=[(Xy_train, "Train"), (Xy_test, "Val")],
        evals_result=evals_result,
    )
  return booster

q_solar_xgboost = {}
q_solar_xgboost["true"] = feature_engineerer_solar.y_test.values
alpha = [x for x in np.arange(0.1, 1.0, 0.1)]
booster = train_xgboost_model(feature_engineerer_solar.X_train, feature_engineerer_solar.y_train, feature_engineerer_solar.X_val, feature_engineerer_solar.y_val, alpha)
# only inplace_predict can work with QuantileDMatrix
scores = booster.inplace_predict(feature_engineerer_solar.X_test)
# dim 1 is the quantiles
# assert scores.shape[0] == feature_engineerer_solar.X_test.shape[0]
# assert scores.shape[1] == alpha.shape[0]

for i in range(len(alpha)):

  q_solar_xgboost[str(alpha[i])] = scores[:, i]  # alpha=0.05


[0]	Train-quantile:44.13515	Val-quantile:92.39949
[1]	Train-quantile:42.29184	Val-quantile:88.13729
[2]	Train-quantile:40.54912	Val-quantile:84.14312
[3]	Train-quantile:38.89848	Val-quantile:80.30528
[4]	Train-quantile:37.31716	Val-quantile:76.64144
[5]	Train-quantile:35.80335	Val-quantile:73.11867
[6]	Train-quantile:34.35909	Val-quantile:69.76359
[7]	Train-quantile:32.98317	Val-quantile:66.56863
[8]	Train-quantile:31.66048	Val-quantile:63.54328
[9]	Train-quantile:30.40100	Val-quantile:60.65396
[10]	Train-quantile:29.19980	Val-quantile:57.91469
[11]	Train-quantile:28.06596	Val-quantile:55.29548
[12]	Train-quantile:26.97471	Val-quantile:52.83934
[13]	Train-quantile:25.94129	Val-quantile:50.50737
[14]	Train-quantile:24.95708	Val-quantile:48.31265
[15]	Train-quantile:24.01307	Val-quantile:46.22179
[16]	Train-quantile:23.12180	Val-quantile:44.20001
[17]	Train-quantile:22.27242	Val-quantile:42.36014
[18]	Train-quantile:21.46098	Val-quantile:40.56213
[19]	Train-quantile:20.68992	Val-quantile

In [68]:
q_solar_xgboost_df = pd.DataFrame(q_solar_xgboost)

In [71]:
q_solar_xgboost_df.tail(20)

,true,0.1,0.2,0.30000000000000004,0.4,0.5,0.6,0.7000000000000001,0.8,0.9
6403,807.979685,634.287048,679.174622,706.835815,695.908264,693.813477,713.976990,726.605469,729.565430,739.876770
6404,738.905630,579.498413,619.247864,625.399597,634.617004,626.495911,654.884155,661.676636,656.842834,703.941223
6405,641.360705,534.451904,562.789185,575.463623,585.085632,573.680481,602.074036,631.556335,615.869568,634.204407
6406,540.135685,403.834290,441.319092,476.473114,487.358765,486.819031,503.422089,520.712891,488.804596,521.545654
6407,428.443677,333.458374,384.722351,383.259460,383.926239,402.846161,400.217834,391.133759,419.679718,435.357300
6408,301.218273,227.709702,265.213989,265.328094,283.517975,282.360748,280.506012,306.482849,307.939026,315.221527
6409,185.929933,162.050064,173.060516,170.241974,172.885483,171.980499,182.647842,182.846756,189.040161,206.706650
6410,104.851963,89.966171,94.827042,95.940079,101.795486,99.705681,100.505905,97.382278,97.526039,102.517906
6411,66.543140,49.818935,50.400543,59.359718,57.846905,59.304489,66.969261,62.040714,64.515831,68.726234
6412,43.347884,28.628231,32.576309,35.745342,37.998264,45.012180,41.763668,42.742386,47.089081,54.149120


In [72]:
pinball_score(q_solar_xgboost_df, quantiles=alpha)

6.622343823371301

### __Lightgbm implementation__

In [80]:
import lightgbm as lgb

from sklearn.linear_model import QuantileRegressor

quantiles = [x for x in np.arange(0.1, 1.0, 0.4)]
qr_solar_lightgbm = {}
qr_solar_lightgbm["true"] = feature_engineerer_solar.y_test.values
#out_bounds_predictions = np.zeros_like(y_true_mean, dtype=np.bool_)
for quantile in quantiles:
    qr_lightgbm = lgb.LGBMRegressor(objective='quantile', alpha=quantile)
    qr_lightgbm.fit(feature_engineerer_solar.X_train, feature_engineerer_solar.y_train)
    qr_solar_lightgbm[str(quantile)] = qr_lightgbm.predict(feature_engineerer_solar.X_test)

qr_solar_lightgbm_df = pd.DataFrame(qr_solar_lightgbm)
pinball_score(qr_solar_lightgbm_df, quantiles=quantiles)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002978 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5040
[LightGBM] [Info] Number of data points in the train set: 48168, number of used features: 27
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003294 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5040
[LightGBM] [Info] Number of data points in the train set: 48168, number of used features: 27
[LightGBM] [Info] Start training from score 0.030927
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003282 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5040
[LightGBM] [Info] Number of data points in the train set: 48168, number of used features: 27
[LightGBM] [Info] Start training from score 423.432465


8.86896635965327

In [117]:
feature_engineerer_xgboost = Preprocessing_test.FeatureEngineerer(merged_pes, label = 'Solar_MWh_credit')


merged_pes_simple = merged_pes[['solar_down_rad', 'Solar_MWh_credit', 'Wind_MWh_credit']]
feature_engineerer_baseline = Preprocessing_test.FeatureEngineerer(merged_pes_simple, label = 'Solar_MWh_credit')

In [118]:
import model_utils
import importlib
importlib.reload(model_utils)

quantiles = np.arange(0.1, 1.0, 0.1)

model_save_dir_xgboost = "xgboost_models"

xgboost_model = model_utils.XGBoostModel(feature_engineerer_xgboost, quantiles=quantiles, model_save_dir=model_save_dir_xgboost, load_pretrained=False)
xgboost_model.train_and_predict()  # This will skip training if the model is already loaded
print(f"XGBoost Pinball Score: {xgboost_model.pinball_score()}")

[0]	Train-quantile:44.13515	Val-quantile:92.39949
[1]	Train-quantile:42.29184	Val-quantile:88.13729
[2]	Train-quantile:40.54912	Val-quantile:84.14312
[3]	Train-quantile:38.89848	Val-quantile:80.30528
[4]	Train-quantile:37.31716	Val-quantile:76.64144
[5]	Train-quantile:35.80335	Val-quantile:73.11867
[6]	Train-quantile:34.35909	Val-quantile:69.76359
[7]	Train-quantile:32.98317	Val-quantile:66.56863
[8]	Train-quantile:31.66048	Val-quantile:63.54328
[9]	Train-quantile:30.40100	Val-quantile:60.65396
[10]	Train-quantile:29.19980	Val-quantile:57.91469
[11]	Train-quantile:28.06596	Val-quantile:55.29548
[12]	Train-quantile:26.97471	Val-quantile:52.83934
[13]	Train-quantile:25.94129	Val-quantile:50.50737
[14]	Train-quantile:24.95708	Val-quantile:48.31265
[15]	Train-quantile:24.01307	Val-quantile:46.22179
[16]	Train-quantile:23.12180	Val-quantile:44.20001
[17]	Train-quantile:22.27242	Val-quantile:42.36014
[18]	Train-quantile:21.46098	Val-quantile:40.56213
[19]	Train-quantile:20.68992	Val-quantile

In [119]:
preds = xgboost_model.predict(feature_engineerer_solar.X_test)

In [120]:
pd.DataFrame(xgboost_model.q_predictions).tail(15)

,true,0.1,0.2,0.30000000000000004,0.4,0.5,0.6,0.7000000000000001,0.8,0.9
6408,301.218273,227.709702,265.213989,265.328094,283.517975,282.360748,280.506012,306.482849,307.939026,315.221527
6409,185.929933,162.050064,173.060516,170.241974,172.885483,171.980499,182.647842,182.846756,189.040161,206.706650
6410,104.851963,89.966171,94.827042,95.940079,101.795486,99.705681,100.505905,97.382278,97.526039,102.517906
6411,66.543140,49.818935,50.400543,59.359718,57.846905,59.304489,66.969261,62.040714,64.515831,68.726234
6412,43.347884,28.628231,32.576309,35.745342,37.998264,45.012180,41.763668,42.742386,47.089081,54.149120
6413,18.449292,12.748090,17.225080,21.264603,24.102768,29.192059,29.285698,31.204857,37.790062,39.361832
6414,1.436979,6.377642,5.783247,14.159158,14.227736,14.256230,13.412308,11.348673,19.170111,16.528851
6415,0.000000,0.219444,0.219444,0.219444,0.219444,0.219444,0.219444,0.219444,0.219444,0.219449
6416,0.000000,0.219444,0.219444,0.219444,0.219444,0.219444,0.219444,0.219444,0.219444,0.219449
6417,0.000000,0.219444,0.219444,0.219444,0.219444,0.219444,0.219444,0.219444,0.219444,0.219449


In [121]:
quantiles = np.arange(0.1, 1.0, 0.1)

# Specify model save directory
model_save_dir_qr = "quantile_regression_models"

qr_model = model_utils.QuantileRegressorModel(feature_engineerer_baseline, quantiles, model_save_dir=model_save_dir_qr, load_pretrained=False)
qr_model.train_and_predict()  # This will skip training for already loaded models
print(f"Quantile Regressor Pinball Score: {qr_model.pinball_score()}")

Saved Quantile Regressor model for quantile 0.1 to quantile_regression_models\qr_model_quantile_0.1.pkl
Saved Quantile Regressor model for quantile 0.2 to quantile_regression_models\qr_model_quantile_0.2.pkl
Saved Quantile Regressor model for quantile 0.30000000000000004 to quantile_regression_models\qr_model_quantile_0.30000000000000004.pkl
Saved Quantile Regressor model for quantile 0.4 to quantile_regression_models\qr_model_quantile_0.4.pkl
Saved Quantile Regressor model for quantile 0.5 to quantile_regression_models\qr_model_quantile_0.5.pkl
Saved Quantile Regressor model for quantile 0.6 to quantile_regression_models\qr_model_quantile_0.6.pkl
Saved Quantile Regressor model for quantile 0.7000000000000001 to quantile_regression_models\qr_model_quantile_0.7000000000000001.pkl
Saved Quantile Regressor model for quantile 0.8 to quantile_regression_models\qr_model_quantile_0.8.pkl
Saved Quantile Regressor model for quantile 0.9 to quantile_regression_models\qr_model_quantile_0.9.pkl
Qu